In [1]:
from scipy import *
import numpy as np
from pylab import *
from scipy.interpolate import interp1d
import os
from astropy.io import fits
from scipy.special import factorial
%matplotlib inline

In [ ]:
######## check TT, Pk (z=0), Clkk (z=1) from class vs camb

# LCDM parameters
A_s = 2.1e-9
h=0.7
OmegaB = 0.046
OmegaM = 0.3
n_s = 0.97
tau = 0.054 ## only for primary CMB, not use this for simplicity

# beyond LCDM, not implemented yet
OmegaDE = 0.75
w0=-1.0#-0.5#
wa=0.0#0.1#
Neff = 3.046
mnu = 0.0

### derived parameters
#z_reio = 10.0 #11.357
ombh2 = OmegaB*h**2
omch2 = (OmegaM-OmegaB)*h**2
H0=h*100
### accuracy parameters
lmax=5000

In [ ]:
##########################
####### class ############
##########################

## https://github.com/lesgourg/class_public/blob/master/explanatory.ini
# 1.a) list of output spectra requested:
# - 'tCl' for temperature Cls,
# - 'pCl' for polarization Cls,
# - 'lCl' for CMB lensing potential Cls,
# - 'nCl' (or 'dCl') for density number count Cls,
# - 'sCl' for galaxy lensing potential Cls,
# - 'mPk' for total matter power spectrum P(k) infered from gravitational potential,
# - 'dTk' (or 'mTk') for density transfer functions for each species,
# - 'vTk' for velocity transfer function for each species.

# import classy module
from classy import Class
# create instance of the class "Class"
LambdaCDM = Class()
LambdaCDM.set({'omega_b':ombh2,'omega_cdm':omch2,'h':h,'A_s':A_s,'n_s':n_s})
LambdaCDM.set({'output':'tCl,pCl,lCl,mPk,sCl',
               #'lensing':'yes',
               'lensing':'no',
               'P_k_max_1/Mpc':10.0,
               'z_pk':0,
               'reio_parametrization':'reio_none',
               #'z_reio':10.0, 'tau_reio':tau
               'l_switch_limber':100,
               'selection':'dirac',
               'selection_mean':1.0,
               'l_max_lss':lmax,
               'perturb_sampling_stepsize':0.01})
# run class
LambdaCDM.compute()

In [ ]:
si8=LambdaCDM.sigma8()

In [ ]:
# get all C_l output
#cls = LambdaCDM.lensed_cl(2500)
cls = LambdaCDM.raw_cl()
print cls.viewkeys()
ll = cls['ell'][2:]
clTT = cls['tt'][2:]
#clEE = cls['ee'][2:]
#clPP = cls['pp'][2:]

In [ ]:
# get P(k) at redhsift z=0
kk = np.logspace(-4,np.log10(3),1000) # k in h/Mpc
Pk = [] # P(k) in (Mpc/h)**3
h = LambdaCDM.h() # get reduced Hubble for conversions to 1/Mpc
for k in kk:
    Pk.append(LambdaCDM.pk(k*h,0.)*h**3) # function .pk(k,z)

In [ ]:
# get Clkk
cls2=LambdaCDM.density_cl(2500)
print cls2.viewkeys()
ell=cls2['ell'][2:]
clphiphi=cls2['ll'][0][2:]
#clkk=1.0/4 * ((l+2.0)!/(l-2.0)!)* clphiphi
#clkk=1.0/4 * (factorial(ell+2.0)/factorial(ell-2.0))*clphiphi
clkk=1.0/4 * (ell+2.0)*(ell+1.0)*(ell)*(ell-1.0)*clphiphi
#print type(clkk)

In [ ]:
##########################
####### camb #############
##########################

import camb
from camb import model, initialpower
print('Using CAMB %s installed at %s'%(camb.__version__,os.path.dirname(camb.__file__)))

#Set up a new set of parameters for CAMB
pars = camb.CAMBparams()
#This function sets up CosmoMC-like settings, with one massive neutrino and helium set using BBN consistency
pars.set_cosmology(H0=h*100, ombh2=ombh2, omch2=omch2, omk=0, 
                   tau=0, mnu=mnu, neutrino_hierarchy='normal')
### dark energy ##########
pars.set_dark_energy(w=w0, wa=wa, dark_energy_model='fluid') 

pars.InitPower.set_params(As=A_s, ns=n_s, r=0)
pars.set_matter_power(redshifts=[0.,], kmax=10.0)
pars.set_for_lmax(lmax, lens_potential_accuracy=0);
pars.SourceWindows = [camb.sources.GaussianSourceWindow(redshift=1.0, source_type='lensing', sigma=0.0001),]

#calculate results for these parameters
results = camb.get_results(pars)
powers =results.get_cmb_power_spectra(pars, CMB_unit='K')
for name in powers: print(name)

In [ ]:
## primary CMB
unlensedCL=powers['unlensed_scalar'][:,0]
ls = np.arange(unlensedCL.shape[0])

## Pk
kcamb, zcamb, Pkcamb = results.get_matter_power_spectrum(minkh=1e-3, maxkh=10, npoints = 200)
print zcamb,kcamb.shape, Pkcamb.shape

## Clkk
cls_Wz = results.get_source_cls_dict()
for name in cls_Wz: print(name)
clkk_camb = cls_Wz['W1xW1'][2:]
ls2=  np.arange(2, len(clkk_camb)+2)
#print type(clkk_camb),type(ls2)

In [ ]:
# plot C_l^TT
figure(1, figsize=(6,3))
xscale('log');plt.yscale('linear');plt.xlim(2,2500)
xlabel(r'$\ell$',fontsize=16)
ylabel(r'$[\ell(\ell+1)/2\pi]  C_\ell^\mathrm{TT}$',fontsize=16)
plot(ll,clTT*ll*(ll+1)/2./pi,'g-',lw=6,alpha=0.5,label='class')
plot(ls,unlensedCL/2.725**2,'b-',label='camb')
legend(loc=0,frameon=0)
title('2019/4/4 TT comparison (tau=0)')

In [ ]:
# plot P(k)
figure(2, figsize=(6,3))
xscale('log');plt.yscale('log');plt.xlim(kk[0],kk[-1])
xlabel(r'$k \,\,\,\, [h/\mathrm{Mpc}]$',fontsize=16)
ylabel(r'$P(k) \,\,\,\, [\mathrm{Mpc}/h]^3$',fontsize=16)
plot(kk,Pk,'g-',lw=6,alpha=0.5,label='class')
plot(kcamb,Pkcamb[0],'b-',label='camb')
title('2019/4/4 Pk comparison')
legend(loc=0,frameon=0)

In [ ]:
#print type(ell),ell.shape, clkk.shape,ls2.shape, clkk_camb.shape

In [ ]:
# plot C_l^TT
figure(3, figsize=(6,3))
xscale('log');yscale('log');#xlim(2,2500)
xlabel(r'$\ell$',fontsize=16)
ylabel(r'$[\ell(\ell+1)/2\pi]  C_\ell^\mathrm{kk}$',fontsize=16)
plot(ell,clkk*ell*(ell+1)/2./pi,'g-',lw=6,alpha=0.5,label='class')
plot(ls2, clkk_camb,'b-',label='camb')
legend(loc=0,frameon=0)
title('2019/4/4 Clkk comparison (z_s=1)')

In [ ]:
######################################
############## HMF ###################
######################################

mbins = arange(11, 15.5, 0.1)

## murray
import hmf
import astropy.units as u
HMF = hmf.MassFunction(cosmo_params={"Ob0":OmegaB, "H0":H0, "Om0":OmegaM, 
       "m_nu":array([0,0,0])*u.eV},z=0, sigma_8=si8,
        Mmin=11, Mmax=15.5, dlog10m=0.1, n=n_s, delta_wrt='mean',
       hmf_model=hmf.fitting_functions.Tinker08)

from colossus.cosmology import cosmology as cosmo
from colossus.lss import mass_function
cosmo.setCosmology('myCosmo',{'flat': True, 'H0': H0, 'Om0': OmegaM, 
                              'Ob0': OmegaB, 'sigma8': si8, 'ns': n_s,'Ode0':OmegaDE,
                              'de_model':'w0wa','w0':w0,'wa':wa})
# mfunc_so = mass_function.massFunction(10**mbins, 0.0, mdef = '200c', model = 'tinker08',q_out = 'dndlnM')
# mfunc_fof = mass_function.massFunction(10**mbins, 0.0, mdef = '200c', model = 'watson13',q_out = 'dndlnM')
mfunc_so = mass_function.massFunction(10**mbins, 0.0, mdef = 'vir', model = 'tinker08',q_out = 'dndlnM')
mfunc_fof = mass_function.massFunction(10**mbins, 0.0, mdef = 'fof', model = 'watson13',q_out = 'dndlnM')

In [ ]:
figure(4, figsize=(6,3))
xscale('log');yscale('log');#xlim(2,2500)

plot(10**mbins, mfunc_so/h**3,'g-',lw=6,alpha=0.5,label='colossus(SO)')
plot(10**mbins, mfunc_fof/h**3,'r-',lw=2,alpha=1,label='colossus(FOF)')
plot(10**mbins, HMF.dndlog10m,'b--',lw=2,label='Murray')
xlabel(r'$M (M_\odot/h)$',fontsize=16)
ylabel(r'$\frac{dn} {d\log_{10}M} \; (h^3/{\rm Mpc}^3)$',fontsize=16)
legend(loc=0,frameon=0)
title('2019/4/4 HMF comparison (z=0)')